[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fbsamples/sdstudio/blob/main/Diffusion_Studio_0_9.ipynb)


Copyright (c) Meta Platforms, Inc. and affiliates. Author: Nickolai Kniazev

#Instructions (TL;DR) 🎨

1. Click on `open in Colab` link above.
    - Once you get your colab notebook running, you can simple execute `Run All` or press `⌘/ctrl+F9`.
    - All necessary packages will be downloaded at the beginning of the notebook and it may take ~1 minute.
    - [Optional] You'll be invited to grant access to link with google drive to be able to save pictures and to share them. 
2. Copy generated IP address after "Copy IP address on the next line" 
3. Click the link on the bottom (which looks like `***.loca.lt`)
   - Past IP address to the endpoint box
   - [Optional] In order use Stable Diffusion 1.4 and Stable Diffusion 1.4 on MidJournery:
      *  Register on hugging face to get access token. 
      *  Copy your huggingface token to clipboard and past it to the left menu.

In [ ]:
print("...Loading libraries it should take about 35 seconds...")

!pip install -q diffusers==0.10.2 transformers scipy ftfy nevergrad scikit-learn streamlit==1.16.0 streamlit-drawable-canvas &> /dev/null
!npm install --silent -q localtunnel &> /dev/null
!pip install -q git+https://github.com/sberbank-ai/Real-ESRGAN.git -q gfpgan protobuf==3.20.3 rasterio &> /dev/null
!mkdir GFPGAN -p; wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P GFPGAN &> /dev/null
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1zn80TVA4FhIWzSsE26EQ_pjMw3PKkC7u' -O d_studio.py &> /dev/null

import joblib, threading, time, os, pydrive, oauth2client, google, pydrive.auth, pydrive.drive, requests, json, uuid
try:
    google.colab.auth.authenticate_user()
    gauth = pydrive.auth.GoogleAuth()
    gauth.credentials = oauth2client.client.GoogleCredentials.get_application_default()
    drive = pydrive.drive.GoogleDrive(gauth)

    def save_and_create(file_path: str, drive):
        file = drive.CreateFile(metadata={"title": f"{file_path[:-4]}{uuid.uuid1()}.png", 'mimeType': 'image/jpeg'})
        file.SetContentFile(file_path)
        file.Upload()
        res = requests.post(f"https://www.googleapis.com/drive/v3/files/{file['id']}/permissions?supportsAllDrives=true", 
                            data=json.dumps( {'type': 'anyone', 'value': 'anyone', 'role': 'reader'}),
                            headers={'Authorization': 'Bearer ' + gauth.credentials.access_token, 
                                     'Content-Type': 'application/json'})
        return f"https://www.facebook.com/sharer/sharer.php?u=https://drive.google.com/uc?id={file['id']}"

    class TestThreading(object):
        def __init__(self, interval=1, links_dict={}):
            self.interval = interval
            self.kill = False
            self.links_dict = links_dict
            self.thread = threading.Thread(target=self.run, args=())
            self.thread.daemon = True
            self.thread.start()

        def run(self):
            while not self.kill :
                time.sleep(self.interval )
                new_dict = {d : save_and_create(d,drive) for d in os.listdir()
                             if "image_" in d and ".png" in d and d not in self.links_dict.keys()}
                self.links_dict.update(new_dict)
                joblib.dump(self.links_dict,"links_dict.j") if 0 < len(new_dict) else None

    tr = TestThreading(1)
    print("Images will be saved on google Drive")
except:
    print("Access to Google Drive was not granted")

print("Copy IP adress on the next line:")
!streamlit run /content/d_studio.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

In [ ]:
!streamlit run /content/d_studio.py &>/content/logs.txt

In [ ]:
npx localtunnel --port 8501